In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import torchvision
import os
import time

In [3]:
from models import FireClassifier, BACKBONES, transform
from utils import accuracy_gpu

# Load saved model

In [4]:
weight_path = "../weights/resnet50-epoch-1-val_acc=0.99-test_acc=-1.00.pt"
device = torch.device("cuda:0")

model = torch.load(weight_path)
model = model.to(device)
model.eval()

FireClassifier(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
  

# Define datasets

In [5]:
dataset_paths = {
    "afd_train": "/media/tomek/BIG2/datasets/FIRE/aerial_fire_dataset/train",
    "afd_test": "/media/tomek/BIG2/datasets/FIRE/aerial_fire_dataset/test/",
    "dunnings_train": "/media/tomek/BIG2/datasets/FIRE/dunnings/fire-dataset-dunnings/images-224x224/train",
    "dunnings_test": "/media/tomek/BIG2/datasets/FIRE/dunnings/fire-dataset-dunnings/images-224x224/test",
    "combined_train": "/media/tomek/BIG2/datasets/FIRE/combined_dunnings_afd/train",
    "combined_test": "/media/tomek/BIG2/datasets/FIRE/combined_dunnings_afd/test"
}

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.4005, 0.3702, 0.3419), std=(0.2858, 0.2749, 0.2742)),
    ]
)

In [6]:
def average(l):
    return sum(l)/len(l)

# External test set

In [7]:
test_dunnings = torchvision.datasets.ImageFolder(root=dataset_paths['combined_test'],
                                                 transform=transform)

test = torch.utils.data.DataLoader(
    test_dunnings,
    batch_size=8,
    num_workers=0,
    shuffle=True
)

In [8]:
test_acc = []
with torch.no_grad():
    for i, data in enumerate(test):
        inputs = data[0].to(device)
        labels = data[1].to(device)

        outputs = model(inputs)
        pred = outputs.squeeze() > 0.5
        acc = torch.sum(pred == labels).double()/pred.numel()
        acc = float(acc)
        test_acc.append(acc)
        print(f'testing batch {i}/{len(test)} batch accuracy: {acc:.4f} cumulative: {average(test_acc):.4f}')

testing batch 0/392 batch accuracy: 0.8750 cumulative: 0.8750
testing batch 1/392 batch accuracy: 1.0000 cumulative: 0.9375
testing batch 2/392 batch accuracy: 1.0000 cumulative: 0.9583
testing batch 3/392 batch accuracy: 0.8750 cumulative: 0.9375
testing batch 4/392 batch accuracy: 1.0000 cumulative: 0.9500
testing batch 5/392 batch accuracy: 0.7500 cumulative: 0.9167
testing batch 6/392 batch accuracy: 1.0000 cumulative: 0.9286
testing batch 7/392 batch accuracy: 0.7500 cumulative: 0.9062
testing batch 8/392 batch accuracy: 0.8750 cumulative: 0.9028
testing batch 9/392 batch accuracy: 0.8750 cumulative: 0.9000
testing batch 10/392 batch accuracy: 0.8750 cumulative: 0.8977
testing batch 11/392 batch accuracy: 0.8750 cumulative: 0.8958
testing batch 12/392 batch accuracy: 0.8750 cumulative: 0.8942
testing batch 13/392 batch accuracy: 1.0000 cumulative: 0.9018
testing batch 14/392 batch accuracy: 0.8750 cumulative: 0.9000
testing batch 15/392 batch accuracy: 1.0000 cumulative: 0.9062
te

testing batch 130/392 batch accuracy: 0.8750 cumulative: 0.9332
testing batch 131/392 batch accuracy: 1.0000 cumulative: 0.9337
testing batch 132/392 batch accuracy: 1.0000 cumulative: 0.9342
testing batch 133/392 batch accuracy: 0.8750 cumulative: 0.9338
testing batch 134/392 batch accuracy: 1.0000 cumulative: 0.9343
testing batch 135/392 batch accuracy: 0.8750 cumulative: 0.9338
testing batch 136/392 batch accuracy: 0.8750 cumulative: 0.9334
testing batch 137/392 batch accuracy: 0.6250 cumulative: 0.9312
testing batch 138/392 batch accuracy: 0.8750 cumulative: 0.9308
testing batch 139/392 batch accuracy: 1.0000 cumulative: 0.9313
testing batch 140/392 batch accuracy: 0.8750 cumulative: 0.9309
testing batch 141/392 batch accuracy: 0.8750 cumulative: 0.9305
testing batch 142/392 batch accuracy: 1.0000 cumulative: 0.9309
testing batch 143/392 batch accuracy: 0.8750 cumulative: 0.9306
testing batch 144/392 batch accuracy: 1.0000 cumulative: 0.9310
testing batch 145/392 batch accuracy: 1.

testing batch 259/392 batch accuracy: 1.0000 cumulative: 0.9380
testing batch 260/392 batch accuracy: 0.8750 cumulative: 0.9377
testing batch 261/392 batch accuracy: 0.8750 cumulative: 0.9375
testing batch 262/392 batch accuracy: 0.8750 cumulative: 0.9373
testing batch 263/392 batch accuracy: 1.0000 cumulative: 0.9375
testing batch 264/392 batch accuracy: 0.7500 cumulative: 0.9368
testing batch 265/392 batch accuracy: 1.0000 cumulative: 0.9370
testing batch 266/392 batch accuracy: 1.0000 cumulative: 0.9373
testing batch 267/392 batch accuracy: 0.8750 cumulative: 0.9370
testing batch 268/392 batch accuracy: 0.8750 cumulative: 0.9368
testing batch 269/392 batch accuracy: 0.8750 cumulative: 0.9366
testing batch 270/392 batch accuracy: 1.0000 cumulative: 0.9368
testing batch 271/392 batch accuracy: 0.8750 cumulative: 0.9366
testing batch 272/392 batch accuracy: 0.7500 cumulative: 0.9359
testing batch 273/392 batch accuracy: 1.0000 cumulative: 0.9361
testing batch 274/392 batch accuracy: 1.

testing batch 389/392 batch accuracy: 0.8750 cumulative: 0.9394
testing batch 390/392 batch accuracy: 0.8750 cumulative: 0.9393
testing batch 391/392 batch accuracy: 1.0000 cumulative: 0.9394


In [9]:
average(test_acc)

0.9394132653061225

# AFD test set

In [10]:
test_afd = torchvision.datasets.ImageFolder(root=dataset_paths['afd_test'],
                                            transform=transform)

test_afd.class_to_idx = {'positive': 1, 'negative': 0} # class mapping
test = torch.utils.data.DataLoader(
    test_afd,
    batch_size=8,
    num_workers=0,
    shuffle=True
)

In [11]:
test_acc = []
with torch.no_grad():
    for i, data in enumerate(test):
        inputs = data[0].to(device)
        labels = data[1].to(device)

        outputs = model(inputs)
        pred = ~ (outputs.squeeze() > 0.5)
        acc = torch.sum(pred == labels).double()/pred.numel()
        acc = float(acc)
        test_acc.append(acc)
#         break
        print(f'testing batch {i}/{len(test)} batch accuracy: {acc:.4f} cumulative: {average(test_acc):.4f}')

testing batch 0/26 batch accuracy: 0.8750 cumulative: 0.8750
testing batch 1/26 batch accuracy: 1.0000 cumulative: 0.9375
testing batch 2/26 batch accuracy: 0.8750 cumulative: 0.9167
testing batch 3/26 batch accuracy: 1.0000 cumulative: 0.9375
testing batch 4/26 batch accuracy: 1.0000 cumulative: 0.9500
testing batch 5/26 batch accuracy: 1.0000 cumulative: 0.9583
testing batch 6/26 batch accuracy: 1.0000 cumulative: 0.9643
testing batch 7/26 batch accuracy: 1.0000 cumulative: 0.9688
testing batch 8/26 batch accuracy: 1.0000 cumulative: 0.9722
testing batch 9/26 batch accuracy: 0.8750 cumulative: 0.9625
testing batch 10/26 batch accuracy: 0.8750 cumulative: 0.9545
testing batch 11/26 batch accuracy: 1.0000 cumulative: 0.9583
testing batch 12/26 batch accuracy: 1.0000 cumulative: 0.9615
testing batch 13/26 batch accuracy: 1.0000 cumulative: 0.9643
testing batch 14/26 batch accuracy: 1.0000 cumulative: 0.9667
testing batch 15/26 batch accuracy: 1.0000 cumulative: 0.9688
testing batch 16/2